In [16]:
import xarray as xr
import numpy as np
import xarray as xr
import proplot as pplt
from eofs.standard import Eof
import os
from datetime import datetime
import pandas as pd

In [17]:
def linear_reg_slope(x, y):
    pf = np.polyfit(x, y, 1)
    #print(pf)
    return xr.DataArray(pf[0])

def linear_reg_intercep(x, y):
    pf = np.polyfit(x, y, 1)
    #print(pf)
    return xr.DataArray(pf[1])


In [18]:
#Load in ERA5 input data
download_path='/home/jupyter-dipti/work/processed/'
targetpath='isosph5.nonorm.ERA5_Exp8_Input.nc'
InputFile=download_path+targetpath
Indata=xr.open_dataset(InputFile)
Indata
lon=Indata.lon
lat=Indata.lat

varList=list(Indata.keys())
nonormVars = ['cres_nonorm','crel_nonorm','netTOAcs_nonorm','netSurfcs_nonorm','cresSurf_nonorm','crelSurf_nonorm']#[s for s in varList if "_nonorm" in s]
otherVarList=[s for s in varList if not "_nonorm" in s]
nonormVars
otherVarList.remove('lsMask')
#otherVarList


In [19]:
#Change this cell to be either Jan or July
Jan=[1]
July=[7]
Indata=Indata.where(Indata.time.dt.month.isin(Jan), drop=True)
Indata

<xarray.Dataset>
Dimensions:           (time: 63, ncells: 10242)
Coordinates:
  * time              (time) datetime64[ns] 1959-01-01 1960-01-01 ... 2021-01-01
    lon               (ncells) float64 ...
    lat               (ncells) float64 ...
Dimensions without coordinates: ncells
Data variables: (12/23)
    clivi_pre         (time, ncells) float64 -0.7302 0.3835 ... -1.127 -1.441
    clivi             (time, ncells) float32 0.02115 0.02934 ... 0.005324
    clwvi_pre         (time, ncells) float64 0.3943 1.033 ... -0.5825 -0.114
    clwvi             (time, ncells) float32 0.00627 0.09986 ... 0.07535 0.07649
    cres_pre          (time, ncells) float64 -0.7881 -0.6655 ... 0.113 0.8462
    cres              (time, ncells) float32 -1.87 -119.5 ... -57.79 -45.46
    ...                ...
    netSurfcs_nonorm  (time, ncells) float32 5.185 8.301 ... -3.497 -5.719
    crelSurf_nonorm   (time, ncells) float32 -0.4592 0.418 ... -0.2762 -0.19
    cresSurf_nonorm   (time, ncells) float32 -0.01136 -6.653 ... 1.438 4.239
    netTOAcs_nonorm   (time, ncells) float32 0.5978 0.2293 ... -1.699 -2.529
    cres_nonorm       (time, ncells) float32 -0.06596 -6.288 ... 1.492 4.313
    crel_nonorm       (time, ncells) float32 -0.03375 1.004 ... -6.244 -7.278
Attributes:
    CDI:          Climate Data Interface version 2.0.4 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Thu Sep 15 00:34:19 2022: cdo remapbil,/home/jupyter-haruki...
    CDO:          Climate Data Operators version 2.0.4 (https://mpimet.mpg.de...

In [20]:
## load ENSO data
txtPath='/home/jupyter-dipti/work/AiBEDO_simultaneousPreds_Salva/Data/'
inDataFile='BESTEnso.txt'
indexData=pd.read_table(txtPath+inDataFile,skiprows=2,delim_whitespace=True,names=['year','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
indexData

indexData.loc[indexData['year'] == 2003]

years=indexData['year'].values
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

ENSOInd=[]
#date=[]
for year in years:
    for inm,m in enumerate(months):
        ENSOInd.append(indexData.loc[indexData['year'] == year][m].values[0])
        #date.append(str(year)+str(inm+1)+'01')
    
timeIndex=np.arange(np.datetime64("1870-01"), np.datetime64("2023-01"), np.timedelta64(1, "M")).astype('datetime64[ns]')
timeIndex

daEnso = xr.DataArray(
    data=ENSOInd,
    dims=[ "time"],
    coords=dict(
        
        time=timeIndex,
        
    ),
    attrs=dict(
        description="ENSO index",
        units="https://psl.noaa.gov/people/cathy.smith/best/",
    ),
)

#daEnso.sel(time=slice('1870-01-01','1870-12-01'))


EnsoIndices=daEnso.sel(time=slice('1959-01-01','2021-12-01'))
EnsoIndices=EnsoIndices.where(EnsoIndices.time.dt.month.isin(Jan), drop=True)


#prBiasList     
EnsoHigh=np.mean(EnsoIndices)+2*np.std(EnsoIndices)
print(EnsoHigh)

#prBiasList     
EnsoLow=np.mean(EnsoIndices)-2*np.std(EnsoIndices)
EnsoLow


####Regression
varList=['cres_nonorm','crel_nonorm','netTOAcs_nonorm','cresSurf_nonorm','crelSurf_nonorm','netSurfcs_nonorm']
Data=Indata.drop_vars(otherVarList)
outDataHigh=Data.sel(time=slice('2010-01-01','2010-12-01')).copy()
outDataLow=Data.sel(time=slice('2010-01-01','2010-12-01')).copy()



for i,vname in enumerate(varList):
    
    slope = xr.apply_ufunc(linear_reg_slope,
                        EnsoIndices, Indata[vname],
                        vectorize=True,
                        input_core_dims=[['time'], ['time']],# reduce along 'plev'
                        )
    intercept = xr.apply_ufunc(linear_reg_intercep,
                        EnsoIndices, Indata[vname],
                        vectorize=True,
                        input_core_dims=[['time'], ['time']],# reduce along 'plev'
                        )
    #print(slope)
    #print(intercept)
    outlow=slope*EnsoLow+intercept
    outDataLow[vname].values=np.tile(outlow.values, (1, 1))
    outhigh=slope*EnsoHigh+intercept
    outDataHigh[vname].values=np.tile(outhigh.values, (1, 1))
       
out_pathLow = '/home/jupyter-dipti/work/AiBEDO_simultaneousPreds_Salva/Data/Predictions/ERA5_Input_Jan_ENSO_low.nc'
outDataLow.to_netcdf(path=out_pathLow,mode='w',format='NETCDF4')
 
out_pathHigh = '/home/jupyter-dipti/work/AiBEDO_simultaneousPreds_Salva/Data/Predictions/ERA5_Input_Jan_ENSO_high.nc'
outDataHigh.to_netcdf(path=out_pathHigh,mode='w',format='NETCDF4')



<xarray.DataArray ()>
array(2.25957815)


In [21]:
## load SAM data
txtPath='/home/jupyter-dipti/work/AiBEDO_simultaneousPreds_Salva/Data/'
inDataFile='AAO_SAM.txt'
indexData=pd.read_table(txtPath+inDataFile,skiprows=4,delim_whitespace=True,names=['year','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
indexData

indexData.loc[indexData['year'] == 2003]

years=indexData['year'].values
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']



SAMInd=[]
#date=[]
for year in years:
    for inm,m in enumerate(months):
        SAMInd.append(indexData.loc[indexData['year'] == year][m].values[0])
        #date.append(str(year)+str(inm+1)+'01')
    
timeIndex=np.arange(np.datetime64("1979-01"), np.datetime64("2023-01"), np.timedelta64(1, "M")).astype('datetime64[ns]')
timeIndex


daSAM = xr.DataArray(
    data=SAMInd,
    dims=[ "time"],
    coords=dict(
        
        time=timeIndex,
        
    ),
    attrs=dict(
        description="SAM index",
        units="https://psl.noaa.gov/people/cathy.smith/best/",
    ),
)


SAMIndices=daSAM.sel(time=slice('1979-01-01','2021-12-01'))
SAMIndices=SAMIndices.where(SAMIndices.time.dt.month.isin(Jan), drop=True)


#prBiasList     
SAMHigh=np.mean(SAMIndices)+2*np.std(SAMIndices)
print(SAMHigh)

#prBiasList     
SAMLow=np.mean(SAMIndices)-2*np.std(SAMIndices)
SAMLow


####Regression
varList=['cres_nonorm','crel_nonorm','netTOAcs_nonorm','cresSurf_nonorm','crelSurf_nonorm','netSurfcs_nonorm']
Data=Indata.drop_vars(otherVarList)
outDataHigh=Data.sel(time=slice('2010-01-01','2010-12-01')).copy()
outDataLow=Data.sel(time=slice('2010-01-01','2010-12-01')).copy()

Indata1=Indata.sel(time=slice('1979-01-01','2021-12-01'))

for i,vname in enumerate(varList):
    
    slope = xr.apply_ufunc(linear_reg_slope,
                        SAMIndices, Indata[vname].sel(time=slice('1979-01-01','2021-12-01')),
                        vectorize=True,
                        input_core_dims=[['time'], ['time']],# reduce along 'plev'
                        )
    intercept = xr.apply_ufunc(linear_reg_intercep,
                        SAMIndices, Indata[vname].sel(time=slice('1979-01-01','2021-12-01')),
                        vectorize=True,
                        input_core_dims=[['time'], ['time']],# reduce along 'plev'
                        )
    #print(slope)
    #print(intercept)
    outlow=slope*SAMLow+intercept
    outDataLow[vname].values=np.tile(outlow.values, (1, 1))
    outhigh=slope*SAMHigh+intercept
    outDataHigh[vname].values=np.tile(outhigh.values, (1, 1))
       
out_pathLow = '/home/jupyter-dipti/work/AiBEDO_simultaneousPreds_Salva/Data/Predictions/ERA5_Input_Jan_SAM_low.nc'
outDataLow.to_netcdf(path=out_pathLow,mode='w',format='NETCDF4')

out_pathHigh = '/home/jupyter-dipti/work/AiBEDO_simultaneousPreds_Salva/Data/Predictions/ERA5_Input_Jan_SAM_high.nc'
outDataHigh.to_netcdf(path=out_pathHigh,mode='w',format='NETCDF4')



<xarray.DataArray ()>
array(1.83251924)


In [22]:
## load AO data
txtPath='/home/jupyter-dipti/work/AiBEDO_simultaneousPreds_Salva/Data/'
inDataFile='AO_NAM.txt'
indexData=pd.read_table(txtPath+inDataFile,skiprows=4,delim_whitespace=True,names=['year','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
indexData
indexData.loc[indexData['year'] == 2003]

years=indexData['year'].values
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

AOInd=[]
#date=[]
for year in years:
    for inm,m in enumerate(months):
        AOInd.append(indexData.loc[indexData['year'] == year][m].values[0])
        #date.append(str(year)+str(inm+1)+'01')
    
timeIndex=np.arange(np.datetime64("1950-01"), np.datetime64("2023-01"), np.timedelta64(1, "M")).astype('datetime64[ns]')
timeIndex

daAO = xr.DataArray(
    data=AOInd,
    dims=[ "time"],
    coords=dict(
        
        time=timeIndex,
        
    ),
    attrs=dict(
        description="AO index",
        units="https://psl.noaa.gov/people/cathy.smith/best/",
    ),
)


AOIndices=daAO.sel(time=slice('1959-01-01','2021-12-01'))
AOIndices=AOIndices.where(AOIndices.time.dt.month.isin(Jan), drop=True)


#prBiasList     
AOHigh=np.mean(AOIndices)+2*np.std(AOIndices)
print(AOHigh)

#prBiasList     
AOLow=np.mean(AOIndices)-2*np.std(AOIndices)
AOLow


####Regression
varList=['cres_nonorm','crel_nonorm','netTOAcs_nonorm','cresSurf_nonorm','crelSurf_nonorm','netSurfcs_nonorm']
Data=Indata.drop_vars(otherVarList)
outDataHigh=Data.sel(time=slice('2010-01-01','2010-12-01')).copy()
outDataLow=Data.sel(time=slice('2010-01-01','2010-12-01')).copy()



for i,vname in enumerate(varList):
    
    slope = xr.apply_ufunc(linear_reg_slope,
                        AOIndices, Indata[vname],
                        vectorize=True,
                        input_core_dims=[['time'], ['time']],# reduce along 'plev'
                        )
    intercept = xr.apply_ufunc(linear_reg_intercep,
                        AOIndices, Indata[vname],
                        vectorize=True,
                        input_core_dims=[['time'], ['time']],# reduce along 'plev'
                        )
    #print(slope)
    #print(intercept)
    outlow=slope*AOLow+intercept
    outDataLow[vname].values=np.tile(outlow.values, (1, 1))
    outhigh=slope*AOHigh+intercept
    outDataHigh[vname].values=np.tile(outhigh.values, (1, 1))
       
out_pathLow = '/home/jupyter-dipti/work/AiBEDO_simultaneousPreds_Salva/Data/Predictions/ERA5_Input_Jan_AO_low.nc'
outDataLow.to_netcdf(path=out_pathLow,mode='w',format='NETCDF4')

out_pathHigh = '/home/jupyter-dipti/work/AiBEDO_simultaneousPreds_Salva/Data/Predictions/ERA5_Input_Jan_AO_high.nc'
outDataHigh.to_netcdf(path=out_pathHigh,mode='w',format='NETCDF4')



<xarray.DataArray ()>
array(2.74367482)


In [23]:

## load NAO data
txtPath='/home/jupyter-dipti/work/AiBEDO_simultaneousPreds_Salva/Data/'
inDataFile='NAO.txt'
indexData=pd.read_table(txtPath+inDataFile,skiprows=3,delim_whitespace=True,names=['year','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
indexData

indexData.loc[indexData['year'] == 2003]


years=indexData['year'].values
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

NAOInd=[]
#date=[]
for year in years:
    for inm,m in enumerate(months):
        NAOInd.append(indexData.loc[indexData['year'] == year][m].values[0])
        #date.append(str(year)+str(inm+1)+'01')
    
timeIndex=np.arange(np.datetime64("1950-01"), np.datetime64("2023-01"), np.timedelta64(1, "M")).astype('datetime64[ns]')
timeIndex

daNAO = xr.DataArray(
    data=NAOInd,
    dims=[ "time"],
    coords=dict(
        
        time=timeIndex,
        
    ),
    attrs=dict(
        description="NAO index",
        units="https://psl.noaa.gov/people/cathy.smith/best/",
    ),
)


NAOIndices=daNAO.sel(time=slice('1959-01-01','2021-12-01'))
NAOIndices=NAOIndices.where(NAOIndices.time.dt.month.isin(Jan), drop=True)


#prBiasList     
NAOHigh=np.mean(NAOIndices)+2*np.std(NAOIndices)
print(NAOHigh)

#prBiasList     
NAOLow=np.mean(NAOIndices)-2*np.std(NAOIndices)
NAOLow


####Regression
varList=['cres_nonorm','crel_nonorm','netTOAcs_nonorm','cresSurf_nonorm','crelSurf_nonorm','netSurfcs_nonorm']
Data=Indata.drop_vars(otherVarList)
outDataHigh=Data.sel(time=slice('2010-01-01','2010-12-01')).copy()
outDataLow=Data.sel(time=slice('2010-01-01','2010-12-01')).copy()



for i,vname in enumerate(varList):
    
    slope = xr.apply_ufunc(linear_reg_slope,
                        NAOIndices, Indata[vname],
                        vectorize=True,
                        input_core_dims=[['time'], ['time']],# reduce along 'plev'
                        )
    intercept = xr.apply_ufunc(linear_reg_intercep,
                        NAOIndices, Indata[vname],
                        vectorize=True,
                        input_core_dims=[['time'], ['time']],# reduce along 'plev'
                        )
    #print(slope)
    #print(intercept)
    outlow=slope*NAOLow+intercept
    outDataLow[vname].values=np.tile(outlow.values, (1, 1))
    outhigh=slope*NAOHigh+intercept
    outDataHigh[vname].values=np.tile(outhigh.values, (1, 1))
       
out_pathLow = '/home/jupyter-dipti/work/AiBEDO_simultaneousPreds_Salva/Data/Predictions/ERA5_Input_Jan_NAO_low.nc'
outDataLow.to_netcdf(path=out_pathLow,mode='w',format='NETCDF4')

out_pathHigh = '/home/jupyter-dipti/work/AiBEDO_simultaneousPreds_Salva/Data/Predictions/ERA5_Input_Jan_NAO_high.nc'
outDataHigh.to_netcdf(path=out_pathHigh,mode='w',format='NETCDF4')



<xarray.DataArray ()>
array(2.08921056)
